In [2]:
# imports
import matplotlib.pyplot as plt
import numpy as np
import pickle
import os, sys
from deap import base
from deap import creator
sys.path.insert(0, 'evoman')
from environment import Environment
from demo_controller import player_controller


pygame 2.1.2 (SDL 2.0.18, Python 3.8.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
# Create classes
creator.create("FitnessMax", base.Fitness, weights=(1.0, 1.0, 1.0))
creator.create("Individual", np.ndarray, fitness=creator.FitnessMax)

In [4]:
root = './deap_stats_object_results/'
file_list = []
for i in range(11):
    file = root + 'deap_stats_object_' + str(i)
    file_list.append(file)

In [5]:
def get_pickle_object(file_path: str):
    file = open(file_path,'rb')
    object_file = pickle.load(file)
    return object_file

In [6]:
def get_best_agent(data: dict()) -> list():
    winner = 0
    winner_tuple = (0.0, 0.0, 0.0)
    for key, value in data.items():
        if type(key) == int:
            for individual in value:
                if individual.fitness.values[1] > winner_tuple[1]:
                    winner = individual
                    winner_tuple = individual.fitness.values
    return winner, winner_tuple

In [7]:
best_agents = []
best_tuples = []
for file in file_list:
    data = get_pickle_object(file)
    champion = get_best_agent(data)
    print(champion[1])
    best_agents.append(champion[0])
    best_tuples.append(champion[1])

(90.92814186652396, 57.51679627126235, 57.51679627126235)
(91.02693855665949, 59.896695091941254, 59.896695091941254)
(91.13214184093671, 58.73214184093701, 58.73214184093701)
(91.26693855665948, 62.30669517527581, 62.30669517527581)
(91.39669509194096, 61.75825298494067, 61.75825298494067)
(91.04814186652396, 58.78506424239136, 58.78506424239136)
(91.15936107665803, 59.852892469282864, 59.852892469282864)
(91.09936107665804, 58.70168263345231, 58.70168263345231)
(90.43936107665805, 53.91679627126234, 53.91679627126234)
(91.84784596782228, 68.89669509194117, 68.89669509194117)
(90.93173296930547, 56.911732969305795, 56.911732969305795)


In [ ]:
def champ_champ(agent_list: list(), tuple_list: list()) -> list():
    one_to_rule_them_all = []
    winner = 0
    w_tuple = (0, 0, 0)
    for agent, c_tuple in zip(best_agents, best_tuples):
        if c_tuple > w_tuple:
            w_tuple = c_tuple
            winner = agent
    one_to_rule_them_all = agent
    return one_to_rule_them_all

In [8]:
# choose this for not using visuals and thus making experiments faster
headless = True
if headless:
    os.environ["SDL_VIDEODRIVER"] = "dummy"

In [9]:
experiment_name = 'deap_generalized_agent'
OUTPUT_DIR = './' + experiment_name + '/'
if not os.path.exists(experiment_name):
    os.makedirs(experiment_name)

n_hidden_neurons = 10
# initializes environment with ai player using random controller, playing against static enemy
env = Environment(experiment_name=experiment_name,
                  multiplemode="yes",
                  randomini="yes",
                  clockprec="medium",
                  enemies=[1, 2, 3, 4, 5, 6, 7, 8],
                  playermode="ai",
                  player_controller=player_controller(n_hidden_neurons),
                  enemymode="static",
                  level=2,
                  speed="normal")
n_weights = (env.get_num_sensors()+1)*n_hidden_neurons+(n_hidden_neurons+1)*5
# DEAP parameters
n_objectives = 3
n_partitions = 12

def eval_individual(individual):
    """Function to evaluate current individual """
    fitness, player_life, enemy_life, time = env.play(pcont=individual)
    x1 = fitness
    x2 = (player_life - enemy_life) - np.log(time)
    x3 = player_life - np.log(time)
    return (x1, x2, x3)


MESSAGE: Pygame initialized for simulation.


In [10]:
def plot_individual_gains(data, title, file_name, file_dir='./plots/'):
    plt.title(title)
    box = plt.boxplot(data, patch_artist=True)
    colors = ['mistyrose', 'mistyrose', 'mistyrose']
    for patch, color in zip(box['boxes'], colors):
        patch.set_facecolor(color)
    plt.xticks([1, 2, 3], ['Enemy 1', 'Enemy 4', 'Enemy 8'])
    plt.savefig(file_dir + file_name)
    plt.show()
    plt.close()

In [11]:
def get_individual_gain(best_agents:list()) -> dict():
    my_dict = dict()
    x = 0
    for champion in best_agents:
        results = []
        for i in range(5):
            x1, x2, x3 = eval_individual(champion)
            results.append(x2)
        average = sum(results) / 5
        my_dict[x] = average
        x += 1
    return my_dict

In [12]:
box_dict = get_individual_gain(best_agents)


RUN: run status: enemy: 1; fitness: 22.002787726235887; player life: 0; enemy life: 70.0; time: 148

RUN: run status: enemy: 2; fitness: 91.83715536974009; player life: 68; enemy life: 0; time: 143

RUN: run status: enemy: 3; fitness: 22.51136363026786; player life: 0; enemy life: 70.0; time: 89

RUN: run status: enemy: 4; fitness: 3.950143992750463; player life: 0; enemy life: 90.0; time: 156

RUN: run status: enemy: 5; fitness: 85.68691377820123; player life: 17.20000000000023; enemy life: 0; time: 417

RUN: run status: enemy: 6; fitness: 85.23616819497441; player life: 7.000000000000233; enemy life: 0; time: 236

RUN: run status: enemy: 7; fitness: 39.629361971872335; player life: 0; enemy life: 50.0; time: 215

RUN: run status: enemy: 8; fitness: 90.08661051127251; player life: 56.20000000000032; enemy life: 0; time: 253

RUN: run status: enemy: 1; fitness: 13.195978955266742; player life: 0; enemy life: 80.0; time: 122

RUN: run status: enemy: 2; fitness: 89.9959933129232; player

In [13]:
print(box_dict)
deap_7_8 = []
for key, value in box_dict.items():
    deap_7_8.append(value)

print(deap_7_8)



{0: -9.31264505225215, 1: -12.48711857073819, 2: -16.27247469626503, 3: -13.251676961308789, 4: -36.621860223830495, 5: -18.739029524615404, 6: -21.195062046058773, 7: -20.691883119221295, 8: -15.405183324667547, 9: -16.827175817085816, 10: -19.422494216164772}
